#### Imports

In [3]:
import os
import random

import numpy as np
import cv2 as cv

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from model.load_model import load_model

#### Variables

In [4]:
train_data_path = 'data/train/'
test_data_path = 'data/test/'
image_size = 224

#### Dataset definition

In [33]:
class HotdogDataset(Dataset):
    def __init__(self, data_path: str) -> None:
        data = []
        current_folder = os.path.join(data_path, 'hotdog')
        for file_path in os.listdir(current_folder):
            data.append([file_path, 1])  # 1 for being hotdog
        
        current_folder = os.path.join(data_path, 'nothotdog')
        for file_path in os.listdir(current_folder):
            data.append([file_path, 0])
        
        self.data = np.asarray(data)
        np.random.shuffle(self.data)
        self._n_samples = len(data)
    
    def __len__(self) -> int:
        return self._n_samples
    
    def __getitem__(self, idx: int) -> tuple:
        image, label = self.data[idx]
        image = cv.imread(image)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)
        image = cv.resize(image, (image_size, image_size))
        image = np.moveaxis(image, 2, 0)

        return image, label

#### DataLoaders

In [34]:
train_dataloader = DataLoader(HotdogDataset(train_data_path), batch_size=32, shuffle=True)
test_dataloader = DataLoader(HotdogDataset(test_data_path), batch_size=32, shuffle=True)

#### Model loading

In [35]:
model_params = {
    'model_ver': 4,
    'in_channels': 3,
    'image_size': image_size,
    'num_classes': 2,
    'block_types': ['C', 'C', 'T', 'T'],
}

model = load_model(**model_params)